In [1]:
library(tidyverse)
library(targets)
library(tarchetypes)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.2     ✔ purrr   0.3.4
✔ tibble  3.2.1     ✔ dplyr   1.1.2
✔ tidyr   1.2.1     ✔ stringr 1.4.1
✔ readr   2.1.2     ✔ forcats 0.5.2
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
clusters_tibble = qs::qread('clusters_tibble_class.qs')

clusters_tibble$milo_obj = clusters_tibble %>%
    pull(label) %>%
    paste0("milo_", .) %>%
    rlang::syms()
clusters_tibble$milo_index_obj = clusters_tibble %>%
    pull(label) %>%
    paste0("milo_index_tibble_", .) %>%
    rlang::syms()
clusters_tibble$design_df = clusters_tibble %>%
    pull(label) %>%
    paste0("design_df_", .) %>%
    rlang::syms()
clusters_tibble$mm = clusters_tibble %>%
    pull(label) %>%
    paste0("mm_", .) %>%
    rlang::syms()
clusters_tibble

label,object,day,cluster,milo_obj,milo_index_obj,design_df,mm
<chr>,<list>,<chr>,<chr>,<list>,<list>,<list>,<list>
other,exp_labelled_other,all,other,milo_other,milo_index_tibble_other,design_df_other,mm_other
Day14.other,exp_labelled_other,Day14,other,milo_Day14.other,milo_index_tibble_Day14.other,design_df_Day14.other,mm_Day14.other
Day5.other,exp_labelled_other,Day5,other,milo_Day5.other,milo_index_tibble_Day5.other,design_df_Day5.other,mm_Day5.other
neuron,exp_labelled_neuron,all,neuron,milo_neuron,milo_index_tibble_neuron,design_df_neuron,mm_neuron
Day14.neuron,exp_labelled_neuron,Day14,neuron,milo_Day14.neuron,milo_index_tibble_Day14.neuron,design_df_Day14.neuron,mm_Day14.neuron
Day5.neuron,exp_labelled_neuron,Day5,neuron,milo_Day5.neuron,milo_index_tibble_Day5.neuron,design_df_Day5.neuron,mm_Day5.neuron


In [3]:
contrasts_tibble = tibble(
    contrast_name = c("obob5v5",
                      "obob14v14",
                      "obobBL6",
                      "obobBL6d5",
                      "obobBL6d14",
                      "obob5vBL6",
                      "obob14vBL6",
                      "obobFGF1BL6d5",
                      "obobFGF1BL6d14",
                      "BL6d5d14",
                      "obobVeh_PFd5d14"),
    contrast = c("groupFGF1.Day5.obob - groupVeh_PF.Day5.obob",
                 "groupFGF1.Day14.obob - groupVeh_PF.Day14.obob",
                 "((groupVeh_PF.Day5.obob+groupVeh_PF.Day14.obob)/2) - ((groupVeh_PF.Day5.BL6+groupVeh_PF.Day14.BL6)/2)",
                 "groupVeh_PF.Day5.obob - groupVeh_PF.Day5.BL6",
                 "groupVeh_PF.Day14.obob - groupVeh_PF.Day14.BL6",
                 "groupVeh_PF.Day5.obob - ((groupVeh_PF.Day5.BL6 + groupVeh_PF.Day14.BL6)/2)",
                 "groupVeh_PF.Day14.obob - ((groupVeh_PF.Day5.BL6 + groupVeh_PF.Day14.BL6)/2)",
                 "groupFGF1.Day5.obob - groupVeh_PF.Day5.BL6",
                 "groupFGF1.Day14.obob - groupVeh_PF.Day14.BL6",
                 "groupVeh_PF.Day5.BL6-groupVeh_PF.Day14.BL6",
                 "groupVeh_PF.Day5.obob-groupVeh_PF.Day14.obob")
)
contrasts_tibble

contrast_name,contrast
<chr>,<chr>
obob5v5,groupFGF1.Day5.obob - groupVeh_PF.Day5.obob
obob14v14,groupFGF1.Day14.obob - groupVeh_PF.Day14.obob
obobBL6,((groupVeh_PF.Day5.obob+groupVeh_PF.Day14.obob)/2) - ((groupVeh_PF.Day5.BL6+groupVeh_PF.Day14.BL6)/2)
obobBL6d5,groupVeh_PF.Day5.obob - groupVeh_PF.Day5.BL6
obobBL6d14,groupVeh_PF.Day14.obob - groupVeh_PF.Day14.BL6
obob5vBL6,groupVeh_PF.Day5.obob - ((groupVeh_PF.Day5.BL6 + groupVeh_PF.Day14.BL6)/2)
obob14vBL6,groupVeh_PF.Day14.obob - ((groupVeh_PF.Day5.BL6 + groupVeh_PF.Day14.BL6)/2)
obobFGF1BL6d5,groupFGF1.Day5.obob - groupVeh_PF.Day5.BL6
obobFGF1BL6d14,groupFGF1.Day14.obob - groupVeh_PF.Day14.BL6


In [4]:
da_recipe = tidyr::crossing(clusters_tibble, contrasts_tibble)
da_recipe$label = paste0(da_recipe$label, '___', da_recipe$contrast_name)
head(da_recipe)

label,object,day,cluster,milo_obj,milo_index_obj,design_df,mm,contrast_name,contrast
<chr>,<list>,<chr>,<chr>,<list>,<list>,<list>,<list>,<chr>,<chr>
Day14.neuron___BL6d5d14,exp_labelled_neuron,Day14,neuron,milo_Day14.neuron,milo_index_tibble_Day14.neuron,design_df_Day14.neuron,mm_Day14.neuron,BL6d5d14,groupVeh_PF.Day5.BL6-groupVeh_PF.Day14.BL6
Day14.neuron___obob14v14,exp_labelled_neuron,Day14,neuron,milo_Day14.neuron,milo_index_tibble_Day14.neuron,design_df_Day14.neuron,mm_Day14.neuron,obob14v14,groupFGF1.Day14.obob - groupVeh_PF.Day14.obob
Day14.neuron___obob14vBL6,exp_labelled_neuron,Day14,neuron,milo_Day14.neuron,milo_index_tibble_Day14.neuron,design_df_Day14.neuron,mm_Day14.neuron,obob14vBL6,groupVeh_PF.Day14.obob - ((groupVeh_PF.Day5.BL6 + groupVeh_PF.Day14.BL6)/2)
Day14.neuron___obob5v5,exp_labelled_neuron,Day14,neuron,milo_Day14.neuron,milo_index_tibble_Day14.neuron,design_df_Day14.neuron,mm_Day14.neuron,obob5v5,groupFGF1.Day5.obob - groupVeh_PF.Day5.obob
Day14.neuron___obob5vBL6,exp_labelled_neuron,Day14,neuron,milo_Day14.neuron,milo_index_tibble_Day14.neuron,design_df_Day14.neuron,mm_Day14.neuron,obob5vBL6,groupVeh_PF.Day5.obob - ((groupVeh_PF.Day5.BL6 + groupVeh_PF.Day14.BL6)/2)
Day14.neuron___obobBL6,exp_labelled_neuron,Day14,neuron,milo_Day14.neuron,milo_index_tibble_Day14.neuron,design_df_Day14.neuron,mm_Day14.neuron,obobBL6,((groupVeh_PF.Day5.obob+groupVeh_PF.Day14.obob)/2) - ((groupVeh_PF.Day5.BL6+groupVeh_PF.Day14.BL6)/2)


In [5]:
da_recipe = da_recipe %>%
filter(!(str_detect(contrast_name, "obob5v5") & day == 'Day14')) %>%
filter(!(str_detect(contrast_name, "obob14v14") & day == 'Day5')) %>%
# filter(!(contrast_name == 'obobBL6' & day != 'all')) %>% # this one is good to have, as they may be equivalent
filter(!(contrast_name == 'obobBL6d5' & (day == 'Day14'))) %>%
filter(!(contrast_name == 'obobBL6d14' & (day == 'Day5'))) %>%
filter(!(contrast_name == 'obobFGF1BL6d5' & (day == 'Day14'))) %>% 
filter(!(contrast_name == 'obobFGF1BL6d14' & (day == 'Day5'))) %>%
filter(!(contrast_name == "obob5vBL6" & (day != 'all'))) %>% 
filter(!(contrast_name == "obob14vBL6" & (day != 'all'))) %>%
filter(!(contrast_name == 'BL6d5d14' & (day != 'all'))) %>%
filter(!(contrast_name == 'obobVeh_PFd5d14' & (day != 'all')))

In [6]:
da_recipe

label,object,day,cluster,milo_obj,milo_index_obj,design_df,mm,contrast_name,contrast
<chr>,<list>,<chr>,<chr>,<list>,<list>,<list>,<list>,<chr>,<chr>
Day14.neuron___obob14v14,exp_labelled_neuron,Day14,neuron,milo_Day14.neuron,milo_index_tibble_Day14.neuron,design_df_Day14.neuron,mm_Day14.neuron,obob14v14,groupFGF1.Day14.obob - groupVeh_PF.Day14.obob
Day14.neuron___obobBL6,exp_labelled_neuron,Day14,neuron,milo_Day14.neuron,milo_index_tibble_Day14.neuron,design_df_Day14.neuron,mm_Day14.neuron,obobBL6,((groupVeh_PF.Day5.obob+groupVeh_PF.Day14.obob)/2) - ((groupVeh_PF.Day5.BL6+groupVeh_PF.Day14.BL6)/2)
Day14.neuron___obobBL6d14,exp_labelled_neuron,Day14,neuron,milo_Day14.neuron,milo_index_tibble_Day14.neuron,design_df_Day14.neuron,mm_Day14.neuron,obobBL6d14,groupVeh_PF.Day14.obob - groupVeh_PF.Day14.BL6
Day14.neuron___obobFGF1BL6d14,exp_labelled_neuron,Day14,neuron,milo_Day14.neuron,milo_index_tibble_Day14.neuron,design_df_Day14.neuron,mm_Day14.neuron,obobFGF1BL6d14,groupFGF1.Day14.obob - groupVeh_PF.Day14.BL6
Day14.other___obob14v14,exp_labelled_other,Day14,other,milo_Day14.other,milo_index_tibble_Day14.other,design_df_Day14.other,mm_Day14.other,obob14v14,groupFGF1.Day14.obob - groupVeh_PF.Day14.obob
Day14.other___obobBL6,exp_labelled_other,Day14,other,milo_Day14.other,milo_index_tibble_Day14.other,design_df_Day14.other,mm_Day14.other,obobBL6,((groupVeh_PF.Day5.obob+groupVeh_PF.Day14.obob)/2) - ((groupVeh_PF.Day5.BL6+groupVeh_PF.Day14.BL6)/2)
Day14.other___obobBL6d14,exp_labelled_other,Day14,other,milo_Day14.other,milo_index_tibble_Day14.other,design_df_Day14.other,mm_Day14.other,obobBL6d14,groupVeh_PF.Day14.obob - groupVeh_PF.Day14.BL6
Day14.other___obobFGF1BL6d14,exp_labelled_other,Day14,other,milo_Day14.other,milo_index_tibble_Day14.other,design_df_Day14.other,mm_Day14.other,obobFGF1BL6d14,groupFGF1.Day14.obob - groupVeh_PF.Day14.BL6
Day5.neuron___obob5v5,exp_labelled_neuron,Day5,neuron,milo_Day5.neuron,milo_index_tibble_Day5.neuron,design_df_Day5.neuron,mm_Day5.neuron,obob5v5,groupFGF1.Day5.obob - groupVeh_PF.Day5.obob


In [7]:
qs::qsave(da_recipe, 'da_recipe_class.qs')